In [1]:
import pandas
import pickle

import pandas as pd
import seaborn as sns
import platform
import re
from tqdm import tqdm

In [15]:
YEWNO_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/Yewno/ms_yewno_2020.pickle'
EDF_DATA_PATH = '/Users/khoanguyen/Workspace/dataset/edf_msft/MSFT_Jul2019_2020_linebreak'
FILTERED_YEWNO_PATH = '/Users/khoanguyen/Workspace/dataset/Yewno/filtered_yewno.pickle'

# Windows path
if platform.system() == 'Windows':
    YEWNO_DATA_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\yewno_ms_2020.pickle'
    EDF_DATA_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\Data\\MSFT\\MSFT_Jul2019_2020_linebreak'
    FILTERED_YEWNO_PATH = 'K:\\Lbpam\\DG_Gestion_Quant\\GERANT\\Khoa\\Data\\filtered_yewno.pickle'

In [3]:
with open(YEWNO_DATA_PATH, 'rb') as f:
    yewno_data = pickle.load(f)

In [4]:
with open(EDF_DATA_PATH, 'rb') as f:
    edf_data = pickle.load(f)

In [5]:
yewno_data['Contribution'].sum()

0.9771604537963867

In [6]:
concept_count = []
concept_list = yewno_data['Concept'].tolist()

In [7]:
for concept in tqdm(concept_list):
    # take care of string that contains quantifier, which can cause wonky interaction with re.findall
    if any(x in concept for x in ['+', '#', '.']):
        concept = re.escape(concept)
    # \b is word boundary, which doesn't work with string like 'C++' where there's no word boundary after ++
    re_string = r'(?:^|(?<=\s))' + concept + r'(?=\s|$)'

    # used re.IGNORECASE so that there's no need to convert case for string and text
    concept_per_doc = edf_data['text'].apply(lambda x: len(re.findall(re_string, x, flags=re.IGNORECASE)))
    concept_count.append(concept_per_doc.sum())

100%|██████████████████████████████████████████████████████████████████████████| 16460/16460 [8:32:28<00:00,  1.87s/it]


In [8]:
yewno_data['edf_count'] = concept_count

In [13]:
filtering_results = yewno_data[yewno_data['edf_count'] >= 10]

In [14]:
filtering_results = filtering_results[(filtering_results['Pureplay'] > 0) &
                                      (filtering_results['Contribution'] > 0)]

In [16]:
with open(FILTERED_YEWNO_PATH, 'wb') as f:
    pickle.dump(filtering_results, f)